##  [망고플레이트 크롤링](https://www.mangoplate.com/)    
### [참고 사이트](https://juwon2021.tistory.com/414)   

### ** 시작 전 확인 사항 **
  Window terminal에서   
- conda activate   
- conda activate ml-env   
(ml-env) 환경 확인 후   
- jupyter notebook   
- conda install -c anaconda requests   
- conda install bs4   
- conda install lxml   
- conda install -c conda-forge selenium   
- conda install tqdm   
- connda install -c conda-forge folium   
- connda install -c conda-forge googlemaps   
- pip install chromedriver-autoinstaller   

### 페이지 10개 크롤링 (for문 활용)

In [1]:
# 필요한 라이브러리를 임포트하기

import sys # 시스템
import os  # 시스템

# 데이터 다루기
import pandas as pd
import numpy as np

# selenium 크롤링
from selenium import webdriver  
from selenium.webdriver import ActionChains as AC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# 크롬 드라이버
import chromedriver_autoinstaller

# beautifulsoup 크롤링
import requests
from bs4 import BeautifulSoup

# lxml 크롤링
import lxml.html

# 시간 조절
import time

# 시간 측정
from tqdm import notebook

# 정규표현식
import re

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

from selenium.webdriver.common.keys import Keys

In [17]:
# 음식점 1개에서 정보 크롤링 함수
def FoodCrawling(row, col):
    # 전체 담을 그릇
    total_dict = {}
    k = 0
    
    for i in range(row):
        driver.execute_script("window.scrollTo(0, 400)")
        for j in range(col):
            # 동영상 1개 담을 그릇
            sub_dict = {}
            
            # 음식점 클릭
            element = f'body > main > article > div.column-wrapper > div > div > section > div.search-list-restaurants-inner-wrap > ul > li:nth-child({i+1}) > div:nth-child({j+1}) > figure > a > div > img'
            driver.execute_script("arguments[0].scrollIntoView();", driver.find_element(By.CSS_SELECTOR, element))
            driver.find_element(By.CSS_SELECTOR, element).click()
            # driver.find_element(By.CSS_SELECTOR, '.center-croping lazy').send_keys(Keys.ENTER)
            time.sleep(0.5)

            # 가게 이름 크롤링
            element = f'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.restaurant_title_wrap > span > h1'
            title_raw = driver.find_element(By.CSS_SELECTOR, element)
            title = title_raw.text

            # 가게 전체 평점 점수 크롤링
            element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.restaurant_title_wrap > span > strong > span'
            total_raw = driver.find_element(By.CSS_SELECTOR, element)
            total = total_raw.text
            
            # 조회수 크롤링
            try:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status.branch_none > span.cnt.hit'
                view_raw = driver.find_element(By.CSS_SELECTOR, element)
                view = view_raw.text
            except:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status > span.cnt.hit'
                view_raw = driver.find_element(By.CSS_SELECTOR, element)
                view = view_raw.text

            # 리뷰 개수 크롤링
            try:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status.branch_none > span.cnt.review'
                review_raw = driver.find_element(By.CSS_SELECTOR, element)
                num_review = review_raw.text

            except:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status > span.cnt.review'
                review_raw = driver.find_element(By.CSS_SELECTOR, element)
                num_review = review_raw.text

            # 별표 개수 크롤링
            try:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status.branch_none > span.cnt.favorite'
                star_raw = driver.find_element(By.CSS_SELECTOR, element)
                num_star = star_raw.text

            except:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status > span.cnt.favorite'
                star_raw = driver.find_element(By.CSS_SELECTOR, element)
                num_star = star_raw.text

            # 음식 종류 및 메뉴 크롤링
            try:
                element1 = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child(3) > td > span'
                type_raw = driver.find_element(By.CSS_SELECTOR, element1)
                type_raw = type_raw.text
                food_type = type_raw
            except:
                element1 = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child(2) > td > span'
                type_raw = driver.find_element(By.CSS_SELECTOR, element1)
                type_raw = type_raw.text
                food_type = type_raw
            try:
                element2 = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child(9) > td > ul'
                menu_raw = driver.find_element(By.CSS_SELECTOR, element2)
                food_menu = menu_raw.text
                food_type = [type_raw, food_menu]
            except:
                pass
            
            # sub_dict에 담기
            sub_dict['title'] = title
            sub_dict['Point'] = total
            sub_dict['View'] = view
            sub_dict['Review'] = num_review
            sub_dict['Star'] = num_star
            sub_dict['Type'] = food_type
            
             # total_dict에 담기
            total_dict[k] = sub_dict
            k += 1
            time.sleep(1)
            
            # 뒤로 가기
            driver.back()
            time.sleep(0.5)
            
    df = pd.DataFrame.from_dict(total_dict, orient='index')
    driver.close()
    return df

In [18]:
# 지역 키워드를 입력하기
print('지역 : 강남, 홍대, 이태원')
keyword = input('지역을 입력하시오: ')

지역 : 강남, 홍대, 이태원
지역을 입력하시오: 강남


In [19]:
# 모든 데이터 프레임 담을 그릇
king_dict = {}

# 1페이지부터 순차적으로 들어가 크롤링하기
pages = 10
for page in range(pages):    
    # 크롬창 띄우기
    chrome_path = chromedriver_autoinstaller.install()
    driver = webdriver.Chrome(chrome_path)
    driver.maximize_window() # 창 최대화
    driver.get(" https://www.mangoplate.com/search/{0}?keyword={0}&page={1}".format(keyword, page+1))
    time.sleep(2)
    
    king_dict[page] = FoodCrawling(row=10, col=2)
    time.sleep(0.5)

In [20]:
king_dict

{0:                     title Point     View Review    Star  \
 0                     미라이   4.8  510,477    127  10,338   
 1                    시라카와   4.8  321,374     75   6,227   
 2                     페리지   4.8   52,571     39   1,467   
 3                    상진식감   4.8   30,936     41     878   
 4                    맛짱조개   4.8  130,259     64   3,855   
 5                   스시카나에   4.7  388,309    105   6,100   
 6   BISTROT de YOUNTVILLE   4.7  383,255    156   8,018   
 7                   알라프리마   4.7  254,818     87   5,114   
 8                     마루심   4.7  511,594    222   9,504   
 9                  패티앤베지스   4.7   58,444     88   2,092   
 10                    일리조   4.7   74,899     58   2,000   
 11                  매덕스피자   4.7  107,669     80   3,120   
 12                     까폼   4.7   84,644     69   2,535   
 13                 원디그리노스   4.7   16,061     35     598   
 14                    삐아프   4.7  168,509     85   4,267   
 15                    스시렌   4.7   17

In [21]:
# csv 파일로 저장
df1 = king_dict[0]
df2 = king_dict[1]
df3 = king_dict[2]
df4 = king_dict[3]
df5 = king_dict[4]
df6 = king_dict[5]
df7 = king_dict[6]
df8 = king_dict[7]
df9 = king_dict[8]
df10 = king_dict[9]

df1.to_csv('Gangnam1.csv', encoding='utf-8-sig', index=False)
df2.to_csv('Gangnam2.csv', encoding='utf-8-sig', index=False)
df3.to_csv('Gangnam3.csv', encoding='utf-8-sig', index=False)
df4.to_csv('Gangnam4.csv', encoding='utf-8-sig', index=False)
df5.to_csv('Gangnam5.csv', encoding='utf-8-sig', index=False)
df6.to_csv('Gangnam6.csv', encoding='utf-8-sig', index=False)
df7.to_csv('Gangnam7.csv', encoding='utf-8-sig', index=False)
df8.to_csv('Gangnam8.csv', encoding='utf-8-sig', index=False)
df9.to_csv('Gangnam9.csv', encoding='utf-8-sig', index=False)
df10.to_csv('Gangnam10.csv', encoding='utf-8-sig', index=False)